In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
data["data_atac"] = data["data_rna"]

data["norm_factor_atac"] = data["norm_factor_rna"]

In [4]:
#del(data["data_rna"])
#del( data["norm_factor_rna"])

In [5]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params({"lr" : 0.05,'K': 3, "nb_size_init_rna" : torch.ones(32) * 300,"nb_size_init_atac" : torch.ones(32) * 300,
                            "theta_scale_rna" : torch.ones(32) * 1.5,"theta_scale_atac" : torch.ones(32) * 1.5,
                            "theta_rate_rna" : torch.ones(32) * 0.1, "theta_rate_atac" : torch.ones(32) * 0.1,
                            "norm_init_sd_rna" : torch.ones(32), "norm_init_sd_atac" : torch.ones(32),
                           "probs" : torch.tensor([0.2,0.6,0.2,0.05,0.05]), "likelihood_rna" : "Poisson",
                            "likelihood_atac" : "Gaussian", "lambda" : 0.1})

In [6]:
ll = interface.run(steps = 500)



Running LatentCategorical on 2000 cells with 64 segments for 500 steps and 2 modalities

Done!


ELBO: 50.780472656  : 100%|██████████| 500/500 [00:16<00:00, 30.82it/s]  


In [7]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights_rna': array([1.0000000e+00, 1.7407931e-14, 1.7407931e-14], dtype=float32),
 'mixture_weights_atac': array([1.0000000e+00, 4.5893663e-24, 4.5893663e-24], dtype=float32),
 'segment_factor_rna': array([0.02858344, 0.04018855, 0.01666831, 0.03037301, 0.01906693,
        0.02859587, 0.06827504, 0.01692482, 0.02874506, 0.0572692 ,
        0.01779881, 0.03019137, 0.02843222, 0.03546889, 0.0387943 ,
        0.02119994, 0.0307648 , 0.02131946, 0.0285026 , 0.09181449,
        0.02363962, 0.0404896 , 0.0380477 , 0.01897096, 0.01888343,
        0.03539491, 0.03775993, 0.03098671, 0.02141724, 0.02605571,
        0.00831726, 0.02137379], dtype=float32),
 'NB_size_atac': array([1988.8837  , 2685.9492  , 1230.9305  , 3246.658   , 2059.23    ,
        1597.3435  , 6277.1313  ,   10.069989, 1228.0212  , 4462.859   ,
        1635.5101  ,   11.338124, 2740.1216  , 2639.7864  , 1062.9513  ,
         936.91003 , 1231.0615  ,  656.2137  , 1587.4656  , 7089.8047  ,
        2249.4536  , 1688.

In [6]:

import numpy as np
np.argmax(lr["assignment_probs_rna"], axis = 0)

NameError: name 'lr' is not defined

In [8]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
interface._model.likelihood(lr)

tensor(15.6250)

In [12]:
ll = interface.run(steps = 500, MAP = True)



Running MixtureCategorical on 1000 cells wiht 32 segments for 500 steps

Done!


ELBO: 106373.072000000  : 100%|██████████| 500/500 [00:14<00:00, 35.58it/s]


In [13]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'mixture_weights': array([0.6955218 , 0.30447823], dtype=float32),
 'NB_size': array([6001.4614, 7851.197 , 4123.702 , 6428.5693, 7146.3735, 4104.542 ,
        8943.439 , 3116.494 , 2701.1592, 8803.028 , 7564.94  , 4094.1895,
        6736.6274, 6328.3804, 1413.1351, 6184.788 , 2145.3516, 1346.6396,
        6850.7593, 9435.439 , 7367.5596, 2317.6804, 8416.429 , 6108.8096,
         590.4864, 8129.3286, 7085.8276, 8198.789 , 1302.6709, 8013.4697,
        2689.5356, 3456.9587], dtype=float32),
 'segment_factor': array([13.244415 , 14.819234 , 12.810789 , 14.411754 , 14.229758 ,
        13.376876 , 17.010983 , 10.64102  , 13.214214 , 16.186419 ,
        13.359442 , 15.655713 , 13.376876 , 13.201247 , 14.913476 ,
        10.181025 , 14.277906 , 10.090607 , 13.272441 , 25.42345  ,
        11.412315 , 14.1457815, 13.638575 , 13.828127 , 13.828127 ,
        13.833637 , 13.678261 , 14.23623  , 11.8657055, 12.4854   ,
         6.6277895, 11.452094 ], dtype=float32),
 'CNV_probabilities': array([

In [14]:

import numpy as np
np.argmax(lr["assignment_probs"], axis = 0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
print(np.argmax(lr["CNV_probabilities"][0], axis = 1) + 1)
print(np.argmax(lr["CNV_probabilities"][1], axis = 1) + 1)


[2 3 1 2 1 2 4 1 2 4 1 2 2 2 2 2 2 2 2 4 2 3 2 1 1 2 2 2 1 2 1 1]
[2 2 1 2 1 2 4 2 2 3 1 1 2 3 3 1 2 1 2 4 1 2 3 1 1 3 3 2 2 2 1 2]


In [8]:
interface._model.likelihood(lr)

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray